# Libs

In [2]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.io as pio

pd.options.display.precision = 3
pd.options.display.float_format = lambda x: '%.5f' % x
pd.options.display.max_columns = 15
pd.options.display.max_rows = 6

# Data uploading

In [170]:
df_bal8_v4 = pd.read_csv(r'C:\jupyter\DP\input\logs8_ntd_v5.csv')
def data_preprocessing8():
    df_bal8_v4.columns = df_bal8_v4.columns.str.lower()
    df_bal8_v4.loc[df_bal8_v4.formation=='Balakhany VIII sand','formation'] = '1_bal8_sand'
    df_bal8_v4.loc[df_bal8_v4.formation=='Balakhany VIII 25','formation'] = '2_bal8_25'
    df_bal8_v4.loc[df_bal8_v4.formation=='Balakhany VIII 20','formation'] = '3_bal8_20'
    df_bal8_v4.loc[df_bal8_v4.formation=='Balakhany VIII 15','formation'] = '4_bal8_15'
    df_bal8_v4.loc[df_bal8_v4.formation=='Balakhany VIII 10','formation'] = '5_bal8_10'
    df_bal8_v4.loc[df_bal8_v4.formation=='Balakhany VIII 5','formation'] = '6_bal8_5'
    well_phit_flag8 = df_bal8_v4[df_bal8_v4.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()
    df_bal8_v4_flag = df_bal8_v4[df_bal8_v4.well.isin(well_phit_flag8)]
    return df_bal8_v4_flag
df_bal8_v4_flag = data_preprocessing8()

logs8_ntd_v5 = pd.read_csv(r'C:\jupyter\DP\input\logs8_ntd_v5.csv')
logs8_ntd_v5.field = logs8_ntd_v6.field.str.lower()

ntd_top_phi_bot8_bp_v4 = pd.read_csv(r'C:\jupyter\DP\input\ntd_top_phi_bot8_bp_v4.csv').drop('Unnamed: 0', axis=1)
ntd_top_phi_bot8_bp_v4.columns = ntd_top_phi_bot8_bp_v4.columns.str.lower()

df_bal10_v4 = pd.read_csv(r'C:\jupyter\DP\input\logs10_ntd_v4.csv')
def data_preprocessing10():
    df_bal10_v4.columns = df_bal10_v4.columns.str.lower()
    df_bal10_v4.loc[df_bal10_v4.formation=='Balakhany X sand','formation'] = '1_bal10_sand'
    df_bal10_v4.loc[df_bal10_v4.formation=='Balakhany X 50','formation'] = '2_bal10_40'
    df_bal10_v4.loc[df_bal10_v4.formation=='Balakhany X 40','formation'] = '2_bal10_40'
    df_bal10_v4.loc[df_bal10_v4.formation=='Balakhany X 20','formation'] = '3_bal10_20'
    well_phit_flag10 = df_bal10_v4[df_bal10_v4.phit_flag==1].groupby('well')['phit_flag'].apply(lambda x: x.iloc[0]).reset_index().well.unique()
    df_bal10_v4_flag = df_bal10_v4[df_bal10_v4.well.isin(well_phit_flag10)]
    return df_bal10_v4_flag
df_bal10_v4_flag = data_preprocessing10()

ntd_top_phi_bot10_bp_v4 = pd.read_csv(r'C:\jupyter\DP\input\ntd_top_phi_bot10_bp_v4.csv').drop('Unnamed: 0', axis=1)
ntd_top_phi_bot10_bp_v4.columns = ntd_top_phi_bot10_bp_v4.columns.str.lower()

xy8 = df_bal8_v4_flag[df_bal8_v4_flag.net == 1].groupby('well')[['xmean', 'ymean', 'phit', 'field']].agg(
    {'xmean':'first', 'ymean':'first', 'field':'first', 'phit':'mean'}).reset_index()
xy8 = xy8.rename(columns={'phit':'phit_net_mean'})
xy8 = xy8.round({'xmean':0, 'ymean':0})
xy10 = df_bal10_v4_flag[df_bal10_v4_flag.net == 1].groupby('well')[['xmean', 'ymean', 'phit', 'field']].agg(
    {'xmean':'first', 'ymean':'first', 'field':'first', 'phit':'mean'}).reset_index()
xy10 = xy10.rename(columns={'phit':'phit_net_mean'})
xy10 = xy10.round({'xmean':0, 'ymean':0})

# Display 3d plot

In [4]:
tvdss_top = df_bal8_v4_flag[['well','tst','tvd_scs']].rename(columns={'tst':'layer_top_tst', 'tvd_scs':'layer_top_tvdss'})
tvdss_bot = df_bal8_v4_flag[['well','tst','tvd_scs']].rename(columns={'tst':'layer_bot_tst','tvd_scs':'layer_bot_tvdss'})
field = df_bal8_v4_flag[['well','field']].drop_duplicates().reset_index(drop=True)

In [5]:
df_temp_1 = ntd_top_phi_bot8_bp_v4.set_index(['well','layer_top_tst']).join(tvdss_top.set_index(['well','layer_top_tst']), how='left').reset_index()
df_temp_2 = df_temp_1.set_index(['well','layer_bot_tst']).join(tvdss_bot.set_index(['well','layer_bot_tst']), how='left').reset_index()
df_temp_2 = df_temp_2[[ 'well', 'formation_up', 
                        'phit_avg', 'vsh_avg', 'perm_avg', 'khtst', 'top_htst', 'htst', 'bot_htst',
                        'layer_top_tst', 'layer_bot_tst', 'layer_top_tvdss', 'layer_bot_tvdss',
                        'fm_top_tst', 'fm_bot_tst']]
df_temp_3 = df_temp_2.set_index('well').join(field.set_index('well'), how='left').reset_index()
df_temp_3 = df_temp_3.sort_values(['field','well']).reset_index(drop=True)
df_temp_3['tvdss_center'] = (df_temp_3.layer_top_tvdss + df_temp_3.layer_bot_tvdss) / 2
df_temp_3 = df_temp_3[df_temp_3.htst >= 2 ]
df_temp_3 = df_temp_3.round({'phit_avg':3, 'vsh_avg':3, 'htst':1,
                             'layer_top_tst':0, 'layer_bot_tst':0, 
                             'layer_top_tvdss':0, 'layer_bot_tvdss':0, 'tvdss_center':0})

In [ ]:
df = df_temp_3.copy()
f_list = ['well', 'phit_avg', 'htst', 'tvdss_center']
color = 'field'

def plotly_3d_plot_field(df, features, color_column, output_file):
    # Sample data for the 3D scatter plot
    x = df[features[1]]
    y = df[features[2]]
    z = df[features[3]]
    text = df[features[0]]  # Optional: labels for the points
    categories = df[color_column].unique()

    # Define a dictionary mapping categories to colors
    category_colors = {
        '1_WEST AZERI': 'red',
        '2_CENTRAL AZERI': 'green',
        '3_EAST AZERI': 'blue',
        # Add more categories and colors as needed
    }

    # Create a list to hold the scatter traces
    scatter_traces = []

    for category in categories:
        # Filter data for the current category
        category_data = df[df[color_column] == category]
        
        # Create custom hover text for the current category
        hovertext = [
            f"{features[0]}:{well}<br>{features[1]}:{phit}<br>{features[2]}:{htst}<br>{features[3]}:{vsh}"
            for well, phit, htst, vsh in zip(category_data[features[0]], category_data[features[1]], category_data[features[2]], category_data[features[3]])
        ]

        # Create a Scatter3d object for the current category
        scatter = go.Scatter3d(
            x=category_data[features[1]],
            y=category_data[features[2]],
            z=category_data[features[3]],
            mode='markers',
            marker=dict(
                size=5,
                color=category_colors[category],  # Color by category
                opacity=0.8,
                line=dict(
                    color='black',  # Edge color
                    width=2  # Edge width
                )
            ),
            text=text,  # Labels for the points
            hovertext=hovertext,  # Custom hover text
            hoverinfo='text',  # Use custom hover text
            name=category  # Name for the legend
        )

        # Add the scatter trace to the list
        scatter_traces.append(scatter)

    # Define the layout
    layout = go.Layout(
        title=df['formation_up'].iloc[0],
        scene=dict(
            xaxis_title=features[1],
            # xaxis=dict(
            #     title=features[1],
            #     autorange='reversed'  # Reverse the x-axis
            # ),
            yaxis_title=features[2],
            zaxis=dict(
                title=features[3],
                autorange='reversed'  # Reverse the z-axis
            )
        ),
        width=800,  # Set the width of the plot
        height=600,  # Set the height of the plot
        margin=dict(l=0, r=0, b=0, t=40),
        legend=dict(
            title='Categories',
            itemsizing='constant'
        )
    )

    # Create a Figure object
    fig = go.Figure(data=scatter_traces, layout=layout)
    pio.write_html(fig, file=output_file, auto_open=True)
    # Display the plot
    fig.show()
    
plotly_3d_plot_field(df, f_list, color, 'bal8_3dplot.html')

# Overview tabular data

In [ ]:
fig = plt.figure(figsize=(8,5))
sns.kdeplot(data = ntd_top_phi_bot8_bp_v4[ntd_top_phi_bot8_bp_v4.htst >=2], 
                x='htst', y='bot_htst')
plt.ylim(40,0)
plt.grid()

In [ ]:
ntd_top_phi_bot8_bp_v4

# Matrix plot depth vs phit

In [191]:
def top_phi_bot(dataset):
    df_lst = []
    for wellname in dataset.well.unique()[:]:
        data = dataset[dataset.well == wellname]
        well_lst = []
        phit_lst = []
        htst_lst = []
        bot_lst = []
        vsh_lst = []
        for ind, row in data.iterrows():
            well_lst.append(wellname)
            well_lst.append(wellname)

            phit_lst.append(0)
            phit_lst.append(row['phit_avg'])

            htst_lst.append(row['top_htst'])
            htst_lst.append(row['htst'])
            
            bot_lst.append(row['bot_htst'])

            vsh_lst.append(1)
            vsh_lst.append(row['vsh_avg'])

        phit_lst.append(0)
        htst_lst.append(data['bot_htst'].iloc[-1])
        well_lst.append(wellname)
        well_collect_cluster_short = pd.DataFrame(zip(well_lst, phit_lst, htst_lst, vsh_lst, ), 
                                                  columns=[  'well','phit', 'htst', 'vsh'])
        well_last_row = pd.DataFrame({'well':[well_lst[-1]], 'phit':[0], 'htst': [bot_lst[-1]], 'vsh':[0]})
        well_collect_cluster = pd.concat([well_collect_cluster_short, well_last_row]).reset_index(drop=True)
        well_collect_cluster['depth'] = well_collect_cluster['htst'].cumsum()
        df_lst.append(well_collect_cluster)
    result = pd.concat(df_lst)
    # result['FORMATION_up'] = fm
    return result
data = top_phi_bot(ntd_top_phi_bot8_bp_v4[ntd_top_phi_bot8_bp_v4.htst >= 2])
data.loc[data.well.str.startswith('C'), 'field'] = '1_west_azery'
data.loc[data.well.str.startswith('B'), 'field'] = '2_cent_azery'
data.loc[data.well.str.startswith('D'), 'field'] = '3_east_azery'

In [ ]:
well_lst = []
for well in tqdm(logs8_ntd_v5.well.unique()):
    data = logs8_ntd_v5[logs8_ntd_v5.well == well]
    df_lst = []
    for i in  data.htst_ntd.unique():
        sample = data[data.htst_ntd == i]
        phit_avg = sample.phit.mean()
        sample['phit_ntd'] = phit_avg
        df_lst.append(sample)
    df = pd.concat(df_lst)
    df.loc[df.net == 0, 'phit_ntd'] = 0
    df = df.sort_values('tst')
    well_lst.append(df)
logs8_ntd_v6 = pd.concat(well_lst)
logs8_ntd_v6

In [ ]:
data[(data.well == 'B08Z') & (data.htst >= 2) & (data.phit != 0)].reset_index(drop=True)

In [ ]:
fig, axes = plt.subplots(7, 5, figsize=(9, 17))
axes = axes.flatten()
well_list = logs8_ntd_v6[(logs8_ntd_v6.field == '2_central azeri') & (logs8_ntd_v6.phit_flag == 1)].well.unique()
k = -1
for i, ax in enumerate(axes):
    k = k+1
    if k != len(well_list):
        well = logs8_ntd_v6[logs8_ntd_v6.well == well_list[i]]
        well.loc[well.htst_ntd <= 2, 'phit_ntd'] = 0
        ax.step(well.phit_ntd, well.tst_index, c='g', where='post')
        ax.set_title(well_list[i])
        ax.set_xlim(0, 0.3)
        ax.set_ylim(0, 1500)
        ax.invert_yaxis()
        ax.grid(True)
    else:
        break
plt.tight_layout()
plt.show()
well